In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim as optim
import random
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import os
import time
import yaml
import json
plt.style.use('default')





In [2]:

def set_seed(seed_value=42):
    """Sets the seed for reproducibility in PyTorch, NumPy, and Python."""
    random.seed(seed_value)  # Python random module
    np.random.seed(seed_value) # Numpy module
    torch.manual_seed(seed_value) # PyTorch CPU seeding

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # if you are using multi-GPU.
        # Configure CuDNN for deterministic operations
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        # Optional: Newer PyTorch versions might require this for full determinism
        # Note: This can sometimes throw errors if a deterministic implementation isn't available
        # try:
        #     torch.use_deterministic_algorithms(True)
        # except Exception as e:
        #     print(f"Warning: Could not enable deterministic algorithms: {e}")
        # Optional: Sometimes needed for deterministic matrix multiplication
        # os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

    print(f"Seed set globally to {seed_value}")

In [3]:
# moved to helper_funcs.py for better organization
from helper_funcs import get_data_loaders, get_model, train_model, evaluate


In [4]:
# Load the configuration file with all hyperparameters, model parameters, data paths, etc.
# for tuple: config_tupe_default.yaml
# for time_grid: config_time_grid_default.yaml

with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

config['device'] ="cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"
seed = config["seed"]
set_seed(seed_value=seed)

date = time.strftime("%Y-%m-%d")

# SET MODEL NAME
model_name = f"{config['model_type']}_model_{config['epochs']}_changed_representation_{date}"
config['model_name'] = model_name

print("Model type selected: ", config["model_type"])
print("Model name: ", model_name)

Seed set globally to 42
Model type selected:  tuple
Model name:  tuple_model_30_changed_representation_2025-04-06


## Train Transformers
- Two transformer architectures were used
  - time_grid: for question 3.2a, data is in 49 hour rows with 41 measurement columns
  - tuples: for question3.2b, data is in tuples of (time, measurement, value) and each row is a measurement for a patient


- Load prefered model by either:
- 1. Changing `"model_type"` in config file or
- 2. `config["model_type] = "tuple"  # Choose 'time_grid' or 'tuple'` below

In [5]:
# 1. Load Data
train_loader, val_loader, test_loader = get_data_loaders(config)

# 2. Model Initialization, Criterion, Optimizer
model = get_model(config)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=config["scheduler_patience"])

# 3. Training Loop
best_val_auroc = -1.0
epochs_no_improve = 0

# check if the output directory exists, if not create it
if not os.path.exists(config["output_dir"]):
    os.makedirs(config["output_dir"])
    print(f"Output directory {config['output_dir']} created.")
    
best_model_path = os.path.join(config["output_dir"], f"{config['model_name']}.pth")
num_epoch_trained = 0

for epoch in range(config["epochs"]):
    print(f"\nEpoch {epoch+1}/{config['epochs']}")

    train_loss = train_model(model, train_loader, criterion, optimizer, config["device"], config["model_type"])
    val_loss, val_auroc, val_auprc, _ = evaluate(model, val_loader, criterion, config["device"], config["model_type"])

    scheduler.step(val_auroc)

    if val_auroc > best_val_auroc:
        print(f"Validation AuROC improved ({best_val_auroc:.4f} -> {val_auroc:.4f}). Saving model...")
        best_val_auroc = val_auroc
        torch.save(model.state_dict(), best_model_path)
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"Validation AuROC did not improve. ({epochs_no_improve}/{config['early_stopping_patience']})")

    if epochs_no_improve >= config["early_stopping_patience"]:
            print(f"Early stopping triggered after {epoch + 1} epochs.")
            break
    num_epoch_trained += 1

config["num_epoch_trained"] = num_epoch_trained
#save the config file with the number of epochs trained
config_path = os.path.join(config["output_dir"], f"config_{model_name}.yaml")
with open(config_path, 'w') as f:
    yaml.dump(config, f)

# 4. Load Best Model and Evaluate on Test Set
print("\n--- Loading Best Model for Testing ---")
if os.path.exists(best_model_path):
    model.load_state_dict(torch.load(best_model_path, map_location=config["device"]))

    print("\n--- Evaluating on Test Set ---")
    test_loss, test_auroc, test_auprc, cm = evaluate(model, test_loader, criterion, config["device"], config["model_type"])
    print("\n--- Test Results ---")

else:
    print(f"Warning: Best model file not found at {best_model_path}. Testing with the last state.")
    # Optionally evaluate the final model state if no best model was saved
    print("\n--- Evaluating Last Model State on Test Set ---")
    test_loss, test_auroc, test_auprc, cm = evaluate(model, test_loader, criterion, config["device"], config["model_type"])
    print("\n--- Test Results (Last Epoch Model) ---")


print(f"Test Loss: {test_loss:.4f}")
print(f"Test AuROC: {test_auroc:.4f}")
print(f"Test AuPRC: {test_auprc:.4f}")
print(f"Confusion Matrix:\n{cm}")
print("--------------------")
# Save results to json
results = {
    "test_loss": test_loss,
    "test_auroc": test_auroc,
    "test_auprc": test_auprc,
    "confusion_matrix": cm.tolist(),  # Convert numpy array to list for JSON serialization
}
results_path = os.path.join(config["output_dir"], f"results_{model_name}.json")
with open(results_path, 'w') as f:
    json.dump(results, f, indent=4)


Tuple Data: Train Patients: 4000, Val Patients: 4000, Test Patients: 4000
Using 41 modalities (including padding index 0
Labels - Train: 4000 (Positive: 554), Val: 4000 (Positive: 568), Test: 4000 (Positive: 585)
DataLoaders created.
Model (tuple) created and moved to cuda.

Epoch 1/30


/cluster/courses/ml4h/project1env/lib/python3.12/site-packages/torch/nn/modules/transformer.py:508: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(
                                                           

Validation AuROC improved (-1.0000 -> 0.6821). Saving model...

Epoch 2/30


Validation AuROC improved (0.6821 -> 0.7323). Saving model...

Epoch 3/30


Validation AuROC improved (0.7323 -> 0.7331). Saving model...

Epoch 4/30


...

Validation AuROC improved (0.7886 -> 0.7889). Saving model...

Epoch 28/30


Validation AuROC improved (0.7889 -> 0.7903). Saving model...

Epoch 29/30


Validation AuROC did not improve. (1/7)

Epoch 30/30


Validation AuROC did not improve. (2/7)

--- Loading Best Model for Testing ---

--- Evaluating on Test Set ---



--- Test Results ---
Test Loss: 0.4522
Test AuROC: 0.8039
Test AuPRC: 0.4416
Confusion Matrix:
[[2943  472]
 [ 269  316]]
--------------------
